<a href="https://colab.research.google.com/github/JakeOh/202110_itw_lab_python/blob/main/ml09_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer, load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import plot_roc_curve

# 이진 분류(Binary Class Classification)

* scikit-learn 패키지의 breast cancer 예제 데이터 셋을 로딩.
* 악성종양(malignant)/양성종양(benign): 진짜 암, 암이 아닌 종양을 분류하는 이진 분류 문제.
* 3가지 머신 러닝 분류 모델을 비교
    * KNN
    * Logistic Regression
    * SGD

## Breast cancer datasets 준비

In [2]:
cancer = load_breast_cancer()

In [3]:
print(cancer['DESCR'])

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

In [4]:
# 특성 행렬(2차원 배열), 타겟 배열
X, y = cancer['data'], cancer['target']

In [5]:
X.shape, y.shape

((569, 30), (569,))

In [6]:
cancer['feature_names']  # 특성(변수, 컬럼) 이름들

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

In [7]:
cancer['target_names']

array(['malignant', 'benign'], dtype='<U9')

In [8]:
np.unique(y, return_counts=True)

(array([0, 1]), array([212, 357]))

In [10]:
X[:3, :]

array([[1.799e+01, 1.038e+01, 1.228e+02, 1.001e+03, 1.184e-01, 2.776e-01,
        3.001e-01, 1.471e-01, 2.419e-01, 7.871e-02, 1.095e+00, 9.053e-01,
        8.589e+00, 1.534e+02, 6.399e-03, 4.904e-02, 5.373e-02, 1.587e-02,
        3.003e-02, 6.193e-03, 2.538e+01, 1.733e+01, 1.846e+02, 2.019e+03,
        1.622e-01, 6.656e-01, 7.119e-01, 2.654e-01, 4.601e-01, 1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, 1.326e+03, 8.474e-02, 7.864e-02,
        8.690e-02, 7.017e-02, 1.812e-01, 5.667e-02, 5.435e-01, 7.339e-01,
        3.398e+00, 7.408e+01, 5.225e-03, 1.308e-02, 1.860e-02, 1.340e-02,
        1.389e-02, 3.532e-03, 2.499e+01, 2.341e+01, 1.588e+02, 1.956e+03,
        1.238e-01, 1.866e-01, 2.416e-01, 1.860e-01, 2.750e-01, 8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, 1.203e+03, 1.096e-01, 1.599e-01,
        1.974e-01, 1.279e-01, 2.069e-01, 5.999e-02, 7.456e-01, 7.869e-01,
        4.585e+00, 9.403e+01, 6.150e-03, 4.006e-02, 3.832e-02, 2.058e-02,
        2.250e-02, 4.571e-03, 2.357e

In [11]:
y[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [12]:
y[-10:]

array([1, 1, 1, 0, 0, 0, 0, 0, 0, 1])

## 훈련 셋, 테스트 셋 분리

층화 추출(stratified sampling)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=42)

In [14]:
X_train.shape, X_test.shape

((455, 30), (114, 30))

In [15]:
y_train.shape, y_test.shape

((455,), (114,))

In [23]:
print('Train target:', np.unique(y_train, return_counts=True))
print('Test target:', np.unique(y_test, return_counts=True))

Train target: (array([0, 1]), array([170, 285]))
Test target: (array([0, 1]), array([42, 72]))


## KNN Classifier

In [16]:
# 모델 생성
knn_model = Pipeline(steps=[('scaler', StandardScaler()), 
                            ('clf', KNeighborsClassifier())])

In [17]:
# 모델 훈련
knn_model.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('clf', KNeighborsClassifier())])

In [18]:
# 훈련 셋 예측값
train_pred = knn_model.predict(X_train)

In [19]:
train_pred[:10]

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1])

In [20]:
y_train[:10]

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1])

In [21]:
confusion_matrix(y_train, train_pred)

array([[161,   9],
       [  3, 282]])

In [24]:
# 정확도: 전체 샘플 중에서 양성(암)과 음성(단순 종양)을 정확히 분류한 비율
(161 + 282) / (161 + 9 + 3 + 282)  # (TP + TN) / (TP + FN + FP + TN)

0.9736263736263736

In [25]:
accuracy_score(y_train, train_pred)

0.9736263736263736

In [30]:
# 정밀도: 양성(Positive=1) 예측들 중에서 정답의 비율.
# -> 일반 종양으로 예측한 것들 중에서 진짜 일반 종양들의 비율
282 / (9 + 282)  # TP / (TP + FP)

0.9690721649484536

In [29]:
precision_score(y_train, train_pred)

0.9690721649484536

In [31]:
# 재현율: 실제 양성(Positive=1) 예측들 중에서 정답 비율.
# -> 실제로 일반 종양들 중에서 진짜 일반 종양들의 비율.
282 / (282 + 3)  # TP / (TP + FN)

0.9894736842105263

In [32]:
recall_score(y_train, train_pred)

0.9894736842105263

In [33]:
print(classification_report(y_train, train_pred))

              precision    recall  f1-score   support

           0       0.98      0.95      0.96       170
           1       0.97      0.99      0.98       285

    accuracy                           0.97       455
   macro avg       0.98      0.97      0.97       455
weighted avg       0.97      0.97      0.97       455



In [34]:
# 암의 정밀도: 암으로 예측한 샘플들 중에서 정답 비율
161 / (161 + 3)

0.9817073170731707

In [35]:
# 암의 재현율: 실제 암 샘플들 중에서 정답 비율.
161 / (161 + 9)

0.9470588235294117

In [36]:
# 암의 F1 score
2 / (1/0.98 + 1/0.95)

0.9647668393782384

In [37]:
# 테스트 셋의 예측값
test_pred = knn_model.predict(X_test)

In [38]:
confusion_matrix(y_test, test_pred)

array([[39,  3],
       [ 2, 70]])

In [39]:
print(classification_report(y_test, test_pred))

              precision    recall  f1-score   support

           0       0.95      0.93      0.94        42
           1       0.96      0.97      0.97        72

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114



In [42]:
39 / (39 + 3)

0.9285714285714286

# 다중 클래스 분류(Multi-class Classification)

* scikit-learn 패키지의 iris 예제 데이터 셋을 로딩.
* setosa/versicolor/virginca: 3개 클래스를 갖는 분류 문제.
* 3가지 머신 러닝 분류 모델 비교